In [5]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import mrtrix3, fsl, utility as niu

In [6]:
def init_eddy_wf():
    wf = pe.Workflow(name="eddy_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "bvec_file", "bval_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    return wf

In [7]:
data_dir = os.path.abspath("../data")
layout = BIDSLayout(data_dir)

In [8]:
dwi_files = layout.get(datatype="dwi", extension=["nii.gz", "nii"], return_type="file")

In [9]:
dwi_file = dwi_files[0]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)

In [ ]:
test_wf = init_eddy_wf()
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
inputspec.inputs.bvec_file = bvec_file
inputspec.inputs.bval_file = bval_file

test_wf.write_graph(graph2use="colored")
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True

test_wf.run()